# ClimateBert

by Harsh Vardhan Pachisia



Links used:
1. https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270
2. https://towardsdatascience.com/part-1-data-cleaning-does-bert-need-clean-data-6a50c9c6e9fd
3. https://towardsdatascience.com/does-bert-need-clean-data-part-2-classification-d29adf9f745a
4. https://huggingface.co/climatebert/distilroberta-base-climate-sentiment
5. https://ai.plainenglish.io/bert-pytorch-implementation-prepare-dataset-part-1-efd259113e5a

### 1. Packages and reading in data

In [6]:
!pip install datasets transformers accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00


In [1]:
#importing required packages
import pandas as pd
import torch
import matplotlib.pyplot as plt
import csv

from google.colab import drive
from transformers import pipeline

In [2]:
# setting differences between cuda and cput depending on what device we are on
USE_CUDA = torch.cuda.is_available()

if USE_CUDA:
    DEVICE = torch.device('cuda')
    train_obs = 15000
    test_obs = 1500
    print("Using cuda.")
else:
    DEVICE = torch.device('cpu')
    train_obs = 2000
    test_obs = 200
    print("Using cpu.")

Using cuda.


In [3]:
# Mount GDrive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
# get data
small_data = True
dtype_dict = {'label': int}
data_path = "/content/drive/Shareddrives/adv-ml-project/Data/"
comments = pd.read_csv(data_path + "by_threshold/comments_filtered.csv",
                       quoting=csv.QUOTE_NONNUMERIC, dtype=dtype_dict)

### 2. Preprocessing data for ClimateBert

Doing a test-train split

In [7]:
import random
from datasets import Dataset
#double check seed, we should have it be the same across- taken Santi's seed
random.seed(120938)

In [8]:
def train_test_split(df, percent_test_obs):
  # Identify the observations to assign as test
  num_test_obs = round(df.shape[0] * percent_test_obs)
  ids_test_obs = random.sample(range(df.shape[0]), num_test_obs)
  df['test_split'] = 0
  df.loc[ids_test_obs,'test_split'] = 1

  # Fill NaN values with empty strings, otherwise from_dict will raise an error
  df['body'] = df['body'].fillna('')

  # Creates Dataset from dictionary
  train_dict = {"text": df.loc[df['test_split'] == 0, 'body'].tolist(),
                "label": df.loc[df['test_split'] == 0, 'label'].tolist()}
  test_dict = {"text": df.loc[df['test_split'] == 1, 'body'].tolist(),
               "label": df.loc[df['test_split'] == 1, 'label'].tolist()}
  train_df = Dataset.from_dict(train_dict)
  test_df = Dataset.from_dict(test_dict)

  return train_df, test_df

# Create train and test data
train_data, test_data = train_test_split(comments, 0.2)

In [9]:
# Creating a small training dataset for faster training times
small_train_dataset = train_data.shuffle(seed=42).select(
    [i for i in list(range(train_obs))])
small_test_dataset = test_data.shuffle(seed=42).select(
    [i for i in list(range(test_obs))])

In [ ]:
print(small_train_dataset[10])

{'text': 'And climate change is a good example of why I lean liberal, but as far as I can tell moderates also consider that a huge issue.', 'label': 1}


For pre-processing, we are going to use the distilroberta climate sentiment tokenizer: https://huggingface.co/climatebert/distilroberta-base-climate-sentiment

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.48k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [17]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)



In [22]:

#if using small development data or larger full model
if small_data:
    tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
    tokenized_test = small_test_dataset.map(preprocess_function, batched=True)
else:
    tokenized_train = train_data.map(preprocess_function, batched=True)
    tokenized_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
#pad the data
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

##3. Climate Bert model

https://huggingface.co/climatebert/distilroberta-base-climate-sentiment

### Training the model

In [ ]:
from transformers import AutoModelForSequenceClassification

# need to add num_labels = 2 (since we only have 2 labels)
# since climate bert sentiment has 3 labels, the ignore_mismatched_sizes is
# required. By finetuning/training our model, this will be resolved.

model = AutoModelForSequenceClassification.from_pretrained(
    "climatebert/distilroberta-base-climate-sentiment",
    num_labels=2, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Compute metrics: Taken from class

In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
from transformers import TrainingArguments, Trainer

output_name = data_path + "climatebert_output"

training_args = TrainingArguments(
   output_dir=output_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=3,
   weight_decay=0.01,
   save_strategy="epoch",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.313100
1000,0.253200
1500,0.276400
2000,0.246400
2500,0.194700


TrainOutput(global_step=2814, training_loss=0.24900728129459426, metrics={'train_runtime': 1792.7343, 'train_samples_per_second': 25.101, 'train_steps_per_second': 1.57, 'total_flos': 5068711844142144.0, 'train_loss': 0.24900728129459426, 'epoch': 3.0})

Compute evaluation metrics

In [ ]:
trainer.evaluate()

<ipython-input-42-dcbaed1ca091>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'eval_loss': 0.5046602487564087,
 'eval_accuracy': 0.84,
 'eval_f1': 0.8265895953757225,
 'eval_runtime': 22.2247,
 'eval_samples_per_second': 67.492,
 'eval_steps_per_second': 4.23,
 'epoch': 3.0}

#### Saving the model

In [ ]:
# Use below when on Google Colab.
model_save_directory = "/content/drive/Shareddrives/adv-ml-project/SavedModels/climate_bert_simple_model"
from google.colab import drive
drive.mount("/content/drive")
!mkdir $model_save_directory

# To save a fine-tuned model
trainer.save_model(model_save_directory)

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/Shareddrives/adv-ml-project/SavedModels/climate_bert_simple_model’: File exists


In [ ]:
# To reload a saved model

#from transformers import DistilBertForQuestionAnswering, DistilBertTokenizerFast
#from google.colab import drive
#drive.mount('/content/drive')
#tokenizer = DistilBertTokenizerFast.from_pretrained(checkpoint)
#model = DistilBertForQuestionAnswering.from_pretrained(model_save_directory)

## Create Climate Bert Sentiment Models over different subreddits

In [ ]:
### To-do
# get the subsets of the data, pre-process (tokenize), run models on each

### Getting the datasets

In [10]:
sub_comments = pd.read_csv(data_path + "by_category/comments_filtered.csv", quoting=csv.QUOTE_NONNUMERIC, dtype=dtype_dict)

In [11]:
# Define date
sub_comments['date'] = pd.to_datetime(sub_comments['date'])
sub_comments['year'] = sub_comments['date'].dt.to_period('Y').dt.to_timestamp()
sub_comments['year_month'] = sub_comments['date'].dt.to_period('M').dt.to_timestamp()

In [12]:
# Subset data by subreddit
australia_comments = sub_comments[sub_comments['subreddit.name']=='australia'].reset_index(drop=True)
canada_comments = sub_comments[sub_comments['subreddit.name']=='canada'].reset_index(drop=True)
collapse_comments = sub_comments[sub_comments['subreddit.name']=='collapse'].reset_index(drop=True)
futurology_comments = sub_comments[sub_comments['subreddit.name']=='futurology'].reset_index(drop=True)
donald_comments = sub_comments[sub_comments['subreddit.name']=='the_donald'].reset_index(drop=True)

In [13]:
# Obtain train split for each dataset
australia_train, _ = train_test_split(australia_comments, 0.2)
canada_train, _ = train_test_split(canada_comments, 0.2)
collapse_train, _ = train_test_split(collapse_comments, 0.2)
futurology_train, _ = train_test_split(futurology_comments, 0.2)
donald_train, _ = train_test_split(donald_comments, 0.2)

In [14]:
# Create a smaller training dataset for faster training times
small_australia_train = australia_train.shuffle(seed=42).select([i for i in list(range(train_obs))])
small_canada_train = canada_train.shuffle(seed=42).select([i for i in list(range(train_obs))])
small_collapse_train = collapse_train.shuffle(seed=42).select([i for i in list(range(train_obs))])
small_futurology_train = futurology_train.shuffle(seed=42).select([i for i in list(range(train_obs))])
small_donald_train = donald_train.shuffle(seed=42).select([i for i in list(range(train_obs))])

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-sentiment")

In [19]:
# Tokenize train data
australia_tokenized = small_australia_train.map(preprocess_function, batched=True)
canada_tokenized = small_canada_train.map(preprocess_function, batched=True)
collapse_tokenized = small_collapse_train.map(preprocess_function, batched=True)
futurology_tokenized = small_futurology_train.map(preprocess_function, batched=True)
donald_tokenized = small_donald_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

### Modelling for each subreddit

Model 1: Australia

In [21]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

output_name = data_path + "climatebert_output/australia"

training_args = TrainingArguments(
   output_dir=output_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=3,
   weight_decay=0.01,
   save_strategy="epoch",
)

australia_trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=australia_tokenized,
   eval_dataset=tokenized_test, # Testing on original dataset
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
australia_trainer.train()

In [ ]:
australia_trainer.evaluate()

In [ ]:
# Use below when on Google Colab.
model_save_directory = "/content/drive/Shareddrives/adv-ml-project/SavedModels/climate_bert_australia"
from google.colab import drive
drive.mount("/content/drive")
!mkdir $model_save_directory

# To save a fine-tuned model
trainer.save_model(model_save_directory)

Model 2: Canada

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, ignore_mismatched_sizes=True)

In [ ]:
from transformers import TrainingArguments, Trainer

output_name = data_path + "climatebert_output/canada"

training_args = TrainingArguments(
   output_dir=output_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=3,
   weight_decay=0.01,
   save_strategy="epoch",
)

canada_trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=canada_tokenized,
   eval_dataset=tokenized_test, # Testing on original dataset
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
canada_trainer.train()

In [ ]:
canada_trainer.evaluate()

In [ ]:
# Use below when on Google Colab.
model_save_directory = "/content/drive/Shareddrives/adv-ml-project/SavedModels/climate_bert_canada"
from google.colab import drive
drive.mount("/content/drive")
!mkdir $model_save_directory

# To save a fine-tuned model
trainer.save_model(model_save_directory)

Model 3: Collapse dataset

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, ignore_mismatched_sizes=True)

In [ ]:
from transformers import TrainingArguments, Trainer

output_name = data_path + "climatebert_output/collapse"

training_args = TrainingArguments(
   output_dir=output_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=3,
   weight_decay=0.01,
   save_strategy="epoch",
)

collapse_trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=collapse_tokenized,
   eval_dataset=tokenized_test, # Testing on original dataset
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
collapse_trainer.train()

In [ ]:
collapse_trainer.evaluate()

In [ ]:
# Use below when on Google Colab.
model_save_directory = "/content/drive/Shareddrives/adv-ml-project/SavedModels/climate_bert_collapse"
from google.colab import drive
drive.mount("/content/drive")
!mkdir $model_save_directory

# To save a fine-tuned model
trainer.save_model(model_save_directory)

Model 4: Futurelogy

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, ignore_mismatched_sizes=True)

In [ ]:
from transformers import TrainingArguments, Trainer

output_name = data_path + "climatebert_output/futurology"

training_args = TrainingArguments(
   output_dir=output_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=3,
   weight_decay=0.01,
   save_strategy="epoch",
)

futurology_trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=futurology_tokenized,
   eval_dataset=tokenized_test, # Testing on original dataset
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
futurology_trainer.train()

In [ ]:
futurology_trainer.evaluate()

In [ ]:
# Use below when on Google Colab.
model_save_directory = "/content/drive/Shareddrives/adv-ml-project/SavedModels/climate_bert_futurology"
from google.colab import drive
drive.mount("/content/drive")
!mkdir $model_save_directory

# To save a fine-tuned model
trainer.save_model(model_save_directory)

Model 5: the_donald

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, ignore_mismatched_sizes=True)

In [ ]:
from transformers import TrainingArguments, Trainer

output_name = data_path + "climatebert_output/donald"

training_args = TrainingArguments(
   output_dir=output_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=3,
   weight_decay=0.01,
   save_strategy="epoch",
)

donald_trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=donald_tokenized,
   eval_dataset=tokenized_test, # Testing on original dataset
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
donald_trainer.train()

In [ ]:
donald_trainer.evaluate()

In [ ]:
# Use below when on Google Colab.
model_save_directory = "/content/drive/Shareddrives/adv-ml-project/SavedModels/climate_bert_donald"
from google.colab import drive
drive.mount("/content/drive")
!mkdir $model_save_directory

# To save a fine-tuned model
trainer.save_model(model_save_directory)